Refactoring lost histogram processing file - IT IS RECOVERED!!!

load all packages
load up IA file as dataframe
load up joined shapefile with census info
loop trhough each row, write to a new file, new headers, as well as old, for each new variable create it by saying if a value of a row's column so fow[23] for example is great than or equal to xyz then the new variable is true or false respectively, add new variable value to row, then write all to new csv file
drop unneeded columns
dg loc so its only 50 rows
change price to be a float and without funny symbols
create new variables (that you'll find listed out on the csv file from yesterday)

when you have the new variable in the new csv file, lets load that into qgis and join it again to the census shapefile, watch filenames
then we'll have a new variables, with census data, geolocated file! all in one place! 
then drop more columns that don't need (from census)

In [2]:
import geojson
import geopandas
import pandas as pd
import numpy as np
import statistics as st
import matplotlib as plt
import scipy
from scipy import stats
from scipy.stats import describe
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
import folium
import csv
from IPython.display import display, HTML
from pathlib import Path 
import os
from datetime import date, time, datetime
from dateutil import tz #for dealing with timezone
from geopy import distance
from django.contrib.gis.geos import GEOSGeometry, Point

/Users/stateofplace/.virtualenvs/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
def get_time():
    today = date.today()
    current_time = time(datetime.now().hour, datetime.now().minute, datetime.now().second)
    date_today = datetime.combine(today, current_time)
    new_date_time = str(date_today).replace(" ", "_").replace("-", "_").replace(":", "_")
    return(new_date_time)
print(get_time())
new_date_time = get_time()

2022_05_25_11_06_54


In [4]:
def load_files(ia, ia_old): 
    #load and create dataframes
    ia_df = pd.read_csv(ia)
    ia_df_old = pd.read_csv(ia_old)
    #drop
    ia_df.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds', 'maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)
    #rename columns ? 
    #trim columns 
    #ia_df = ia_df.loc[:50, :]
    ia_df_old.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds','maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)

    #handle shape file joined, we need it for some variable creation, but doens't need to be a airbnb joined one, just the census info is fine
    # census_gdf = geopandas.read_file(census_ia)
    return(ia_df, ia_df_old)
#ia_df_clean_old, census_gdf = load_files("Out_CSV/listings_9_1_2020_prepand.csv", "../thesis_app/airbnb_census_joined.shp")
# "../thesis_app/airbnb_census_joined.shp"
ia_df_clean, ia_df_clean_old = load_files("../scrap_work/florence_listings_jan_27.csv", "Out_CSV/listings_9_1_2020_prepand.csv") #using the shapefile with no airbnb data yet makes more sense

In [5]:
#will only work once i think
def fixpricing_file(dataframe):
    #price adjust
    dataframe['price'] = dataframe['price'].str.replace("$", "").str.replace(',', '')
    dataframe['price'] = pd.to_numeric(dataframe['price'])
    #tests
    # print(dataframe['price']) #no special characters 
    # print(type(dataframe['price'])) #float not string 
    return dataframe

ia_df_clean = fixpricing_file(ia_df_clean) #ia_df_clean 


<ipython-input-5-506b0929a9b0>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe['price'] = dataframe['price'].str.replace("$", "").str.replace(',', '')


In [6]:
def df_to_csv(dataframe):
    filepath = Path('../thesis_app/Out_CSV/ia_1_27_22' + new_date_time + '.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True) 
    dataframe.to_csv(filepath)
    #tests
    print(filepath)
    return csv
df_to_csv(ia_df_clean)   #ia_df_clean ia_df_clean_old

../thesis_app/Out_CSV/ia_1_27_222022_05_25_11_06_54.csv


<module 'csv' from '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/csv.py'>

In [7]:
# ia_df_clean_old.head()
# #s.loc[s['has_liscense'] == 0]
# row_old_df = ia_df_clean_old.loc[ia_df_clean_old['id'] == 24469]

# row_old_df['host_id']
# ia_df_clean.head()
# ia_df_clean['minimum_nights_avg_ntm']


In [47]:

def create_new_variables(cleaned_file_input, file_output, dataframe, dataframe_old):
    #do a lot of processing
    all = []
    with open(cleaned_file_input, 'r') as file:
        with open(file_output, 'w', encoding="utf-8") as newfile:
            csvwriter = csv.writer(newfile,lineterminator='\n')
            csvreader = csv.reader(file)
            #extract headers
            header = next(csvreader)
            new_headers = ['days_rented_ltm', 'rounded_revenue_ltm', 'occupancy_rate_approx', 'is_hotel', 'is_entire', 'many_listings', 'only_1_listing', 'only_2_listings', 'host_florence', 'has_liscense', 'is_instant_bookable', 'dist_duomo', 'is_centro', 'is_gavinana', 'is_isolotto', 'is_rifredi', 'is_campo', 'listing_revenue_exceed_LTR', 'effected_by_policy_1', 'effected_by_policy_2', 'effected_by_policy_3', 'commercial', 'very_likely_commercial', 'tourist_tax', 'unpaid_tourist_tax', 'geom']
            for item in new_headers:
                header.append(item)
            all.append(header)
            for row in csvreader:
                
                min_night_avg = float(row[17]) #minimum_nights_avg_ntm
                name_of_listing = row[4]
                #test to filter out long term and inactive listings len(row[24]) > 0 is saying reviews for last 30days are not null, buthow about if reviews for last year are not 0?
                if int(row[23]) > 0 and min_night_avg < 29:
                    #print("test to see what this is row[23] {}, and its type {} and length {}".format(row[23],type(row[23]), len(row[23])))
                    #index row 
                    listing_row_id = int(row[0])
                    listing_id = int(row[1])
                    #year and month of data scraped
                    date_scraped = row[3]
                    year = date_scraped.split('-')[0]
                    month = date_scraped.split('-')[1]
                    day = date_scraped.split('-')[2]
                    #year started hosting 
                    year_started_hosting = int(row[7].split('-')[0])
                    month_started = int(row[7].split('-')[1])
                    # total global and local listings
                    #TODO: check if this is accurate local v global
                    # global_total_listings = int(row[9]) + int(row[25])
                    # ia_df_clean['host_total_listings_count'] + ia_df_clean['calculated_host_listings_count'] #int(row[9]) + int(row[25]) not great to use df because then my value is a series! 
                    #dist_duomo lat, long 
                    duomo_coords = (43.7731, 11.2560)
                    #listing_coords = lat, long
                    listing_coords = (row[11], row[12]) # ia_df_clean['latitude'], ia_df_clean['longitude']
                    ##dist_duomo = duomo_coords - listing_coords
                    dist_duomo = (distance.distance(duomo_coords, listing_coords).m)
                    # geom = ((row[11], row[12]), srid=4326)
                    # pnt = GEOSGeometry('POINT(5 23)') # WKT 
                    # gedjango is long, lat 
                    coords_wkt = (round(float(row[12]),4) , round(float(row[11]),4) )
                    coords_wkt = Point(coords_wkt)
                    geom = coords_wkt

                    #has_liscense
                    if len(row[25]) > 0: 
                        
                        has_liscense = 1
                    else:
                        has_liscense = 0
                    #is_instant_bookable
                    if row[26] == 't': 
                        is_instant_bookable = 1
                    else:
                        is_instant_bookable = 0
                    #getting info for "host_florence" success
                    if ("Florence" or "Firenze") in row[8]: #row[8] ia_df_clean['host_location']
                        host_florence = 1
                    else:
                        host_florence = 0
                        #print(host_florence)
                    #getting info for "is_centro" success
                    #got better average rent from statistica and square meters for rent burden
                    centro_neighborhood_rent = 1238.70 * 12
                    campo_neighborhood_rent = 1078.00 * 12
                    rif_neighborhood_rent = 954.80 * 12
                    iso_neighborhood_rent = 939.40 * 12
                    gav_neighborhood_rent = 1070.30 * 12
                    if "Centro" in row[10]: #row[10] ia_df_clean['neighbourhood_cleansed']
                        is_centro = 1
                    else:
                        is_centro = 0
                    #is_campo success
                    if "Campo" in row[10]:
                        is_campo = 1
                    else:
                        is_campo = 0
                    #is_rifredi success
                    if "Rifredi" in row[10]:
                        is_rifredi = 1
                    else:
                        is_rifredi = 0
                    if "Isolotto" in row[10]:
                        is_isolotto = 1
                    else:
                        is_isolotto = 0
                    if "Gavinana" in row[10]:
                        is_gavinana = 1
                    else:
                        is_gavinana = 0
                #occupancy_rate_approx
                    #did reviews a year divided by how likely someone gives a review (airbnb data) * avg stay in florence (2.6)
                    review_rate = .50
                    avg_length = 3.2 
                    

                    if len(row[24]) > 0: #if number of reviviews in last month more than 0 then it's active or there was an error
                        # print(row[23])
                        num_reviews = float(row[23]) #number_of_reviews_ltm
                        #test here for long term
                        if min_night_avg > 29:
                            print("min_night_avg > 29: is {} on row {}".format(min_night_avg,listing_row_id))
                        elif min_night_avg > avg_length:
                            avg_days = min_night_avg
                        else:
                            avg_days = avg_length
                        num_bookings = review_rate * num_reviews
                        days_rented_ltm = num_bookings * avg_days
                        #test for days
                        #adjusting to make occupancy rate 70 to be conservative
                        if days_rented_ltm > 255:
                            #print("days_rented_ltm > 365: is {} on row {}".format(days_rented_ltm,listing_row_id))
                            days_rented_ltm = 255
                        tourist_tax = (days_rented_ltm * 3) *2 #3 euro which is lower end, 3 start hotel is 3.80 euro and 2 persons conservative 
                        occupancy_rate_approx = (days_rented_ltm/365) * 100

                        if occupancy_rate_approx > 70.00:
                            print("occupancy_rate_approx > 70: is {} on row {}".format(occupancy_rate_approx,listing_row_id))
                            occupancy_rate_approx = 70.00
                        #revenue #have to strip the string for price then turn into float
                        price_stripped_dollar = row[16].strip('$') #price
                        price_replaced_comma = price_stripped_dollar.replace(',','')
                        new_price = float(price_replaced_comma)
                        if new_price < 1.0:
                            print("new_price is basically zero: is {} on row {}".format(new_price,listing_row_id))
                        

                        #yearly revenue
                        revenue_ltm = days_rented_ltm * new_price
                        rounded_revenue_ltm = round(revenue_ltm, 2)
                        if rounded_revenue_ltm >0.0 and rounded_revenue_ltm < new_price:
                            print("rounded_revenue_ltm < {}: and is {} on row {}".format(new_price, rounded_revenue_ltm,listing_row_id))
                        # print(rounded_revenue_ltm)

                    else:
                        days_rented_ltm = 'n/a'
                        rounded_revenue_ltm = 'n/a'
                        # listing_revenue_exceed_LTR = 'n/a'

                    # getting if revenue from a listing exceeds approximiate LTR rent for the year issue wiht variable being assigned too early not sure
                    if is_centro == 1:
                        if rounded_revenue_ltm == 'n/a':
                            listing_revenue_exceed_LTR = 'n/a'
                        elif rounded_revenue_ltm > centro_neighborhood_rent:
                            listing_revenue_exceed_LTR = 1
                        else:
                            listing_revenue_exceed_LTR = 0
                    elif is_gavinana == 1:
                        if rounded_revenue_ltm == 'n/a':
                            listing_revenue_exceed_LTR = 'n/a'
                        elif rounded_revenue_ltm > gav_neighborhood_rent:
                            listing_revenue_exceed_LTR = 1
                        else:
                            listing_revenue_exceed_LTR = 0
                    elif is_isolotto == 1:
                        if rounded_revenue_ltm == 'n/a':
                            listing_revenue_exceed_LTR = 'n/a'
                        elif rounded_revenue_ltm > iso_neighborhood_rent:
                            listing_revenue_exceed_LTR = 1
                        else:
                            listing_revenue_exceed_LTR = 0
                    elif is_rifredi ==1:
                        if rounded_revenue_ltm == 'n/a':
                            listing_revenue_exceed_LTR = 'n/a'
                        elif rounded_revenue_ltm > rif_neighborhood_rent:
                            listing_revenue_exceed_LTR = 1
                        else:
                            listing_revenue_exceed_LTR = 0
                    elif is_campo == 1:
                        if rounded_revenue_ltm == 'n/a':
                            listing_revenue_exceed_LTR = 'n/a'
                        elif rounded_revenue_ltm > campo_neighborhood_rent:
                            listing_revenue_exceed_LTR = 1
                        else:
                            listing_revenue_exceed_LTR = 0
                    # print(listing_revenue_exceed_LTR)
                    # listing_revenue_exceed_LTR = 'n/a'    
                    #many_listings variable success
                    if  int(row[9]) > 2: # ia_df_clean['host_total_listings_count']
                        many_listings = 1
                    else:
                        many_listings = 0
                    #only_1 success
                    if int(row[9]) == 1:
                        only_1_listing = 1
                    else: 
                        only_1_listing = 0
                    #only_2 success
                    if int(row[9]) == 2:
                        only_2_listings = 1
                    else:
                        only_2_listings = 0
                    #is_entire success 
                    if row[13] == 'Entire home/apt':  # ia_df_clean['room_type']
                        is_entire = 1
                    else:
                        is_entire = 0
                    #is_hotel success
                    if row[13] == 'Hotel room':
                        is_hotel = 1
                    else:
                        is_hotel = 0
            
                    #commercial
                    if dataframe.loc[listing_row_id, 'host_total_listings_count'] > 4:
                        commercial = 1
                    elif host_florence == 0:
                        commercial = 1
                    else:
                        commercial = 0
                    #very_likely_commercial
                    if dataframe.loc[listing_row_id, 'host_total_listings_count'] > 2 and dataframe.loc[listing_row_id, 'instant_bookable'] == 't' and dataframe.loc[listing_row_id, 'room_type']== 'Entire home/apt':
                        very_likely_commercial = 1
                    else:
                        very_likely_commercial = 0

                    #likely unpaid touist tax for non liscenced, tourist tax was calculated by (3*days rented ltm)
                    if has_liscense == 0:
                        unpaid_tourist_tax = tourist_tax
                    #effected_by_policy_x (1-liscense,2- -is entire, 3-2 listing max, 4-day limit 90) #switched 2 and 4 from origingal 
                    if has_liscense == 0:
                        effected_by_policy_1 = 1
                    else: 
                        effected_by_policy_1 = 0
                    if is_entire == 1:
                        effected_by_policy_2 = 1
                    else:
                        effected_by_policy_2 = 0
                    #man_listings is more than 2 
                    if many_listings == 1:
                        effected_by_policy_3 = 1
                    else:
                        effected_by_policy_3 = 0
                    # print(geom)
                    new_variables = [days_rented_ltm, rounded_revenue_ltm, occupancy_rate_approx, is_hotel, is_entire, many_listings, only_1_listing, only_2_listings, host_florence, has_liscense, is_instant_bookable, dist_duomo, is_centro, is_gavinana, is_isolotto, is_rifredi, is_campo, listing_revenue_exceed_LTR, effected_by_policy_1, effected_by_policy_2, effected_by_policy_3, commercial, very_likely_commercial, tourist_tax, unpaid_tourist_tax, geom]
                    for item in new_variables:
                        row.append(item)
                    #adding entire row to all 
                    all.append(row)
                else:
                    # print("either inactive or long term listing so removing {} from file".format(name_of_listing))
                    pass
            #writing all rows to new out file
            csvwriter.writerows(all)
    return csvwriter
create_new_variables('../thesis_app/Out_CSV/ia_1_27_222022_05_19_12_53_06commercial_calc.csv', '../thesis_app/Out_CSV/commericial_days_rented.csv', ia_df_clean, ia_df_clean_old)

new_price is basically zero: is 0.0 on row 8294


In [ ]:
#tests until we get pytest with nbmake to work
# days less than 365
# rev more than price
# occupancy less than 100
# has_license contains some 0
# policy_1 contains some 0



In [41]:
# df = pd.read_csv("../thesis_app/Out_CSV/ia_1_27_222022_05_14_19_13_43_new_variables_encode_test.csv")
df = pd.read_csv("../thesis_app/Out_CSV/commericial_days_rented.csv")
df.drop(columns=["Unnamed: 0","name", "host_name","host_location"], inplace=True)

In [42]:
df.to_csv("../../geodjango_tut/geodjango/world/data/csv_ia/to_migrate_ia_1_27_222022_05_14_19_13_43_new_variables_noname_dropped_loc_encode_geom_point.csv")

In [43]:
df.head()

,id,listing_url,last_scraped,host_id,host_since,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,room_type,...,is_campo,listing_revenue_exceed_LTR,effected_by_policy_1,effected_by_policy_2,effected_by_policy_3,commercial,very_likely_commercial,tourist_tax,unpaid_tourist_tax,geom
0,31840,https://www.airbnb.com/rooms/31840,2021-12-17,380378,2011-02-07,36,Centro Storico,43.77709,11.25216,Private room,...,0,0,1,0,1,1,0,19.2,19.2,POINT (11.2522 43.7771)
1,32120,https://www.airbnb.com/rooms/32120,2021-12-17,99235,2010-03-26,1,Gavinana Galluzzo,43.76157,11.27741,Entire home/apt,...,0,0,1,1,0,0,0,9.6,9.6,POINT (11.2774 43.7616)
2,32180,https://www.airbnb.com/rooms/32180,2021-12-17,13925330,2014-04-05,2,Centro Storico,43.76832,11.24348,Entire home/apt,...,0,0,1,1,0,0,0,9.6,9.6,POINT (11.2435 43.7683)
3,32395,https://www.airbnb.com/rooms/32395,2021-12-17,140170,2010-06-07,5,Centro Storico,43.76268,11.24149,Hotel room,...,0,0,1,0,1,1,0,192.0,192.0,POINT (11.2415 43.7627)
4,39115,https://www.airbnb.com/rooms/39115,2021-12-17,167739,2010-07-15,10,Centro Storico,43.77525,11.25258,Private room,...,0,0,1,0,1,1,0,19.2,19.2,POINT (11.2526 43.7752)


#add valid data for geom column for each listing. 
#latitude, longitude, srid=4326
#concatenate, long, lat, and srid=4326
#x=None, y=None, z=None, srid=None
#location = fromstr(f'POINT({longitude} {latitude})', srid=4326)


In [44]:
df.head()
df.shape



(5004, 51)

In [46]:
df.sum()
df.describe()

<ipython-input-46-cf828f66290a>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.sum()


,id,host_id,host_total_listings_count,latitude,longitude,accommodates,bedrooms,price,minimum_nights_avg_ntm,availability_30,...,is_rifredi,is_campo,listing_revenue_exceed_LTR,effected_by_policy_1,effected_by_policy_2,effected_by_policy_3,commercial,very_likely_commercial,tourist_tax,unpaid_tourist_tax
count,5.004000e+03,5.004000e+03,5004.000000,5004.000000,5004.000000,5004.000000,4817.000000,5004.000000,5004.000000,5004.000000,...,5004.000000,5004.000000,5004.000000,5004.000000,5004.000000,5004.000000,5004.000000,5004.000000,5004.000000,5004.000000
mean,2.615981e+07,9.462851e+07,17.695444,43.773431,11.253832,3.814748,1.507162,98.745204,2.344045,13.624101,...,0.059752,0.083533,0.003597,0.921463,0.827138,0.539768,0.606315,0.365508,118.800060,117.468825
std,1.675484e+07,1.123017e+08,48.824920,0.008627,0.015005,1.926870,0.808489,140.824771,2.426993,9.189022,...,0.237051,0.276714,0.059874,0.269042,0.378165,0.498466,0.488615,0.481620,139.051074,138.610178
min,3.184000e+04,3.321000e+04,0.000000,43.729640,11.160070,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.600000,9.600000
25%,1.201552e+07,1.090760e+07,1.000000,43.768620,11.246695,2.000000,1.000000,54.000000,1.100000,6.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,21.000000,21.000000
50%,2.470503e+07,4.359480e+07,3.000000,43.772100,11.254235,4.000000,1.000000,75.000000,2.000000,14.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,67.200000,67.200000
75%,4.096621e+07,1.468890e+08,10.000000,43.776693,11.260520,4.000000,2.000000,106.000000,2.800000,21.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,163.200000,163.200000
max,5.369512e+07,4.328313e+08,371.000000,43.829490,11.331620,16.000000,12.000000,7200.000000,28.700000,30.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1530.000000,1530.000000


In [13]:
#passing this for now since i did this in qgis  
# def create_geocensus_variables(ia_newvar_file_input, file_output, geodataframe):
# #need the joined gdf with airbnb data for the stats on census because we use the listing row id of the airbnb file which only matches with the gdf because it has airnbb data in it... maybe not necessary these variables at all anyway...we loop through csv file with 51 airbnb lisitings...to match with census info.. 
#     all = []
#     with open(ia_newvar_file_input, 'r') as file:
#         with open(file_output, 'w') as newfile:
#             csvwriter = csv.writer(newfile,lineterminator='\n')
#             csvreader = csv.reader(file)
#             #extract headers
#             header = next(csvreader)
#             new_headers = ['census_tract_id', 'num_residents_census', 'num_households_census', 'res_buildings_census', 'num_bedrooms_census', 'num_household_renters_census', 'num_household_owners_census']
#             for item in new_headers:
#                 header.append(item)
#             all.append(header)
#             for row in csvreader:
#             #index row 
#                 listing_row_id = int(row[0])
            
#                 #census_tract_id_sez2011
#                 census_tract_id = geodataframe.loc[listing_row_id, 'SEZ2011']
#                 #census data points 
#                 columns = ['P1', 'A47', 'A46', 'PF1', 'E3', 'E27', 'SEZ2011']
#                 #number of residents per census tract P1
#                 num_residents_census = (geodataframe.loc[listing_row_id, 'P1'])
#                 #number of households per census tract PF1
#                 num_households_census = (geodataframe.loc[listing_row_id, 'PF1'])
#                 #number of residential buildings per census tract E3
#                 res_buildings_census = (geodataframe.loc[listing_row_id, 'E3'])
#                 # number of bedrooms per census tract E27
#                 num_bedrooms_census = (geodataframe.loc[listing_row_id, 'E27'])
#                 #number of household renters per census tract  A46 
#                 num_household_renters_census = (geodataframe.loc[listing_row_id, 'A46'])
#                 #number of household homeowners per census tract A47
#                 num_household_owners_census = (geodataframe.loc[listing_row_id, 'A47'])
#                 #adding new variable to row for each row in file
                
#                 new_variables = [census_tract_id, num_residents_census, num_households_census, res_buildings_census, num_bedrooms_census, num_household_renters_census, num_household_owners_census]
#                 for item in new_variables:
#                     row.append(item)
#                 #adding entire row to all 
#                 all.append(row)
#             #writing all rows to new out file
#             csvwriter.writerows(all)
#     return csvwriter
# create_geocensus_variables('../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables.csv', '../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables_census.csv', census_gdf)

In [14]:
'''
This program reads two csv files and merges them based on a common key column.
'''
# Read the files into two dataframes.
df1 = pd.read_csv('../choropleth/neigh_sum_all.csv')
# df1 = df1.loc[:, :'count_airbnb_listings']
# print(df1.head())
# print(type(df1))
df2 = pd.read_csv('../choropleth/neigh_sum.csv')

# Merge the two dataframes, using _ID column as key
df3 = pd.merge(df1, df2, on = 'neigh')
df3.set_index('neigh', inplace = True)

# Write it to a new CSV file
df3.to_csv('../choropleth/neigh_sum_all.csv')
df3.head()


<ipython-input-14-af6382272e99>:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'commercial_x', 'effected_by_policy_2_x', 'accommodates_x', 'E27_x', 'effected_by_policy_3_x', 'AGG_COUNT_x', 'bedrooms_x', 'effected_by_policy_1_x', 'A46_x', 'PF1_x', 'A47_x'} in the result is deprecated and will raise a MergeError in a future version.
  df3 = pd.merge(df1, df2, on = 'neigh')


,pol3_per100,pol3_perpop,pol2_per100,pol2_perpop,pol1_per100,pol1_perpop,e27_per100,e27_perpop,e3_per100,e3_perpop,...,commercial_y,effected_by_policy_1_y,effected_by_policy_2_y,effected_by_policy_3_y,A46_y,A47_y,PF1_y,bedrooms_y,accommodates_y,E27_y
neigh,,,,,,,,,,,,,,,,,,,,,
Campo di Marte,38.014042,0.380140,63.289870,0.632899,97.191575,0.971916,52.229000,0.522290,9.876856,0.098769,...,554,969,631,379,23439,91745,128355,1390,3377,138012
Centro Storico,54.619455,0.546195,81.440781,0.814408,93.311627,0.933116,64.953083,0.649531,12.368764,0.123688,...,5409,6878,6003,4026,216898,433192,756920,11032,28792,908452
Gavinana Galluzzo,42.602041,0.426020,67.346939,0.673469,96.428571,0.964286,51.574949,0.515749,14.506785,0.145068,...,236,378,264,167,7875,28489,41273,642,1474,46026
Isolotto Legnaia,28.473804,0.284738,69.248292,0.692483,96.583144,0.965831,49.734449,0.497344,9.062604,0.090626,...,226,424,304,125,17493,48245,71821,619,1568,74915
Rifredi,32.313341,0.323133,61.566707,0.615667,95.226438,0.952264,51.511295,0.515113,6.363814,0.063638,...,459,778,503,264,33247,116992,164191,1121,2758,173625


In [15]:
#for each row in df3 
#create a new column with the value of column 1 / column 2 = rate of listings per population
#create a new column with th evalue of column 3 * 100 = rate of listings per 100 people 
# Generate result using pandas
# example to follow 
# # Creating new dataframe
# initial_data = {'First_name': ['Ram', 'Mohan', 'Tina', 'Jeetu', 'Meera'], 
#                 'Last_name': ['Kumar', 'Sharma', 'Ali', 'Gandhi', 'Kumari'], 
#                 'Marks': [12, 52, 36, 85, 23] }
  
# df = pd.DataFrame(initial_data, columns = ['First_name', 'Last_name', 'Marks'])
  
# # Generate result using pandas
# result = []
# for value in df["Marks"]:
#     if value >= 33:
#         result.append("Pass")
#     elif value < 0 and value > 100:
#         result.append("Invalid")
#     else:
#         result.append("Fail")
       
# df["Result"] = result   
# print(df)
# rate_listings_per_pop = []
# for pop_value in df3["P1"]:
#     for count_air in df3["count_airbnb_listings"]:
#         rate = count_air / pop_value
#         rate_listings_per_pop.append(rate) 
# df3["rate_listings_per_pop"] = rate_listings_per_pop   
# df3.head()
for key in df3:
    # print(key)
    pass
    #so each column is a key value pair, the headres are keys, the rows are values
rate_listings_per_pop = []
for item in df3.items():
    # print(item)
    #this prints each column which is the rows
    # print(item[1])
    pass
for key, value in df3.items():
    # print(key, '->', value)
    pass

    

In [16]:
1/3

0.3333333333333333